### Import Python Libraries and Set Up the Environment

In [157]:
import requests
import base64
import json
import getpass
from openai import AzureOpenAI

client = AzureOpenAI(
    # api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_key=getpass.getpass("Enter your Azure OpenAI API key: "),
    api_version="2024-07-01-preview",
    # azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_endpoint=getpass.getpass("Enter your Azure OpenAI endpoint: ")
)


### Write and define your functions

In [158]:
def get_image() -> bytes:
    response = requests.get("http://localhost:30080/deviceshifu-camera/capture")
    return response.content

def set_plc_output(switch: bool) -> str:
    output = 0
    if switch:
        output = 1
    response = requests.get("http://localhost:30080/deviceshifu-plc/sendsinglebit?rootaddress=Q&address=0&start=0&digit=0&value=" + str(output))
    response = requests.get("http://localhost:30080/deviceshifu-plc/sendsinglebit?rootaddress=Q&address=0&start=0&digit=1&value=" + str(output))
    response = requests.get("http://localhost:30080/deviceshifu-plc/sendsinglebit?rootaddress=Q&address=0&start=0&digit=2&value=" + str(output))
    response = requests.get("http://localhost:30080/deviceshifu-plc/sendsinglebit?rootaddress=Q&address=0&start=0&digit=3&value=" + str(output))
    return response.content

def set_led(num: int) -> str:
    response = requests.post("http://localhost:30080/deviceshifu-led/number", json={"value": num})
    return response.content

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_image",
            "description": "A function that gets an image from the /capture endpoint.",
        },
    },
    {
        "type": "function",
        "function": {
            "name": "set_plc_output",
            "description": "A function that sets the output of the PLC.",
            "parameters": {
                "type": "object",
                "properties": {
                    "switch": {
                        "type": "boolean",
                        "description": "To set the output to True or False",
                    },
                },
                "required": ["switch"],
                "additionalProperties": False,
            }
        },
    },
    {
        "type": "function",
        "function": {
            "name": "set_led",
            "description": "A function that sets the display number of the LED.",
            "parameters": {
                "type": "object",
                "properties": {
                    "num": {
                        "type": "integer",
                        "description": "a number to display on the LED, between 0 and 9999",
                    },
                },
                "required": ["num"],
                "additionalProperties": False,
            }
        },
    },
]

### Function call handler

In [159]:
def handle_function_call(tool_call):
    if tool_call.function.name == "get_image":
        image = get_image()
        with open("image.png", "wb") as f:
            f.write(image)
        return base64.b64encode(image).decode('utf-8')
    elif tool_call.function.name == "set_plc_output":
        arguments = json.loads(tool_call.function.arguments)
        switch = arguments['switch']
        return set_plc_output(switch)
    elif tool_call.function.name == "set_led":
        arguments = json.loads(tool_call.function.arguments)
        num = arguments['num']
        return set_led(num)

### 1. Call GPT to take a picture and describe what's in it

In [161]:
prompt = [
    {
        "role": "system",
        "content": "You are a helpful embodied intelligence assistant. Use the supplied tools to assist the user.",
    },
    {   "role": "user",
        "content": "Hi, can you tell what the camera is seeing?"
    },
]

chat_completion = client.chat.completions.create(
    model="gpt-4o-mini", messages=prompt, tools=tools
)
tool_call = chat_completion.choices[0].message.tool_calls[0]
function_call_output = handle_function_call(tool_call)

prompt.append(chat_completion.choices[0].message)
prompt.append(
    {
          "tool_call_id": tool_call.id,
          "role": "tool",
          "name": tool_call.function.name,
          "content": "success",
    }
)
prompt.append(
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Thanks! Can you tell me what is in the image? Specifically what is the number on the display and does the PLC has 4 output lights on?",
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{function_call_output}",
                },
            },
        ],
    }
)
chat_completion = client.chat.completions.create(
    model="gpt-4o-mini", messages=prompt, tools=tools
)
print(chat_completion.choices[0].message.content)
# remove last prompt
prompt.pop()
prompt.append(
    {
        "role": "user",
        "content": "Thanks! Can you tell me what is in the image? Specifically what is the number on the display and does the PLC has 4 output lights on?"
    },
)
# append the response to the prompt
prompt.append(chat_completion.choices[0].message)

The display shows the number **3999**. As for the PLC, it appears that **only one light is on**; the others are not illuminated.


### 2. Increase the LED display number by 5

In [162]:
# ask the model to increase the display number on the LED by 5
prompt.append(
    {"role": "user", "content": "Can you increase the display number on the LED by 5?"}
)



chat_completion = client.chat.completions.create(
    model="gpt-4o-mini", messages=prompt, tools=tools
)
tool_call = chat_completion.choices[0].message.tool_calls[0]
print(tool_call)
function_call_output = handle_function_call(tool_call)
print(chat_completion.choices[0].message.content)
# append the response to the prompt

prompt.append(chat_completion.choices[0].message)
prompt.append(
    {
          "tool_call_id": tool_call.id,
          "role": "tool",
          "name": tool_call.function.name,
          "content": "success",
    }
)

ChatCompletionMessageToolCall(id='call_BK3bSIH9RwXjnicknZJw6TFS', function=Function(arguments='{"num":4004}', name='set_led'), type='function')
None


### 3. Set the PLC output to true if it's off and false if it's on

In [163]:
# ask the model to turn on the PLC output if it's off and turn it off if it's on
prompt.append(
    {"role": "user", "content": "Can you set the PLC output to true if it doesn't have 4 output lights on and off if it does?"}
)



chat_completion = client.chat.completions.create(
    model="gpt-4o-mini", messages=prompt, tools=tools
)
tool_call = chat_completion.choices[0].message.tool_calls[0]
print(tool_call)
function_call_output = handle_function_call(tool_call)
print(chat_completion.choices[0].message.content)
# append the response to the prompt
prompt.append(chat_completion.choices[0].message)

ChatCompletionMessageToolCall(id='call_BK3bSIH9RwXjnicknZJw6TFS', function=Function(arguments='{"switch":true}', name='set_plc_output'), type='function')
None


In [ ]:
print(prompt)








### Thank you!

Any questions or feedback? Please let me know!

Email: xqin@edgenesis.com